In [10]:
import pyrosetta
import pyrosetta_help as ph

no_optH = False 

ignore_unrecognized_res=False  

load_PDB_components=False  
ignore_waters=True 

extra_options= ph.make_option_string(no_optH=no_optH,
                                  ex1=None,
                                  ex2=None,
                                  mute='all',
                                  ignore_unrecognized_res=ignore_unrecognized_res,
                                  load_PDB_components=load_PDB_components,
                                  ignore_waters=ignore_waters)


# capture to log
logger = ph.configure_logger()
pyrosetta.init(extra_options=extra_options)

PyRosetta-4 2022 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python310.Release 2022.35+release.6079c70fa16855c06a9c2070a7085bf619df3065 2022-08-31T18:05:32] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [2]:
template_pose = pyrosetta.pose_from_pdb('4xyj_17-763_C.clean.pdb')

In [15]:
#  alphafold

# uniprot = 'Q01813' #@param {type:"string"}
# af_pose = ph.pose_from_alphafold2(uniprot)

In [16]:
# steal from template

af_pose = ph.pose_from_file('4xyj_17-763_A.clean.pdb')
fragsets = ph.make_fragment_sets(af_pose)
threaded, threader, threadites = ph.thread(target_sequence=af_pose.sequence(),
                                  template_pose=template_pose,
                                  fragment_sets=fragsets)
ph.steal_ligands(template_pose, threaded)
threaded.dump_pdb('threaded.pdb')

True

In [ ]:
# relax the missing residues

MISSING_RES = ','.join([str(af_pose.pdb_info().pdb2pose('A', i)) for i in range(704, 710)])
print(f'Missing residue resnum start 1: {MISSING_RES}')

res_selector = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector(MISSING_RES)
sel_vector = res_selector.apply(threaded)

scorefxn = pyrosetta.create_score_function("ref2015_cart")

cycles = 3
movemap = pyrosetta.MoveMap()
movemap.set_bb(allow_bb=sel_vector)
movemap.set_chi(allow_chi=sel_vector)

fastrelax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, cycles)
fastrelax.set_movemap(movemap)
#fastrelax.cartesian(True)
fastrelax.apply(threaded)

threaded.dump_pdb('threaded_relaxed.pdb')

Missing residue resnum start 1: 688,689,690,691,692,693


'688,689,690,691,692,693'

True